In [1]:
import json
import numpy as np
import os
import pickle
import tensorflow as tf
from datetime import datetime
from time import time
from typing import Dict

tf.keras.backend.clear_session()
# tf.config.experimental_run_functions_eagerly(True)

np.random.seed(42)

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [2]:
from gnn import *

In [3]:
limit = 1000
training_dir = "/home/rmonge/UPC/TFM/gnn-qm9/data/training"
training_fn = [os.path.join(training_dir, fn) for fn in os.listdir(training_dir)
               if not fn.startswith(".")][:limit]
test_dir = "/home/rmonge/UPC/TFM/gnn-qm9/data/test"
test_fn = [os.path.join(test_dir, fn) for fn in os.listdir(test_dir) if not fn.startswith(".")][:limit]
target = "dipole_moment"

print(f"Training samples: {len(training_fn)}")
print(f"Test samples: {len(test_fn)}")

training = tf.data.Dataset.from_generator(
    **GNNInput.get_data_generator(training_fn, target))\
    .shuffle(len(training_fn), reshuffle_each_iteration=True).batch(1)
test = tf.data.Dataset.from_generator(
    **GNNInput.get_data_generator(test_fn, target))\
    .shuffle(len(test_fn), reshuffle_each_iteration=True).batch(1)

Training samples: 1000
Test samples: 1000


In [4]:
graph_spec, y_spec = training.element_spec

hidden_state_size = 20
# message_size = hidden_state_size + graph_spec.edge_features.shape[-1]  # For ConcatenationMessage
message_size = 30
message_passing_iterations = 5
output_size = 1

In [5]:
model = GNN(hidden_state_size=hidden_state_size, message_size=message_size,
            message_passing_iterations=message_passing_iterations,
            message_passing=EdgeNetMessagePassing,
#             message_passing=ConcatenationMessage,
            output_size=output_size)

In [6]:
learning_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.935e-4, decay_steps=900, end_learning_rate=1.84e-4, power=1.0
)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_schedule)
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsoluteError()]

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

log_name = f"gnn_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
log_dir = f"/home/rmonge/UPC/TFM/ignnition_data/CheckPoint/{log_name}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [7]:
start = time()
loss = model.fit(
    training, epochs=10, steps_per_epoch=1000,
    validation_data=test, validation_freq=1, validation_steps=1000,
    callbacks=[tensorboard_callback], use_multiprocessing=True)
time() - start

Epoch 1/10


/home/rmonge/UPC/TFM/gnn-qm9/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


   1/1000 [..............................] - ETA: 0s - loss: 16.2329 - mean_absolute_error: 4.0290WARNING:tensorflow:From /home/rmonge/UPC/TFM/gnn-qm9/.venv/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


   2/1000 [..............................] - ETA: 4:06 - loss: 13.7738 - mean_absolute_error: 3.6964WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0760s vs `on_train_batch_end` time: 0.4165s). Check your callbacks.


1000/1000 [==============================] - 53s 53ms/step - loss: 2.7168 - mean_absolute_error: 1.2462 - val_loss: 2.2847 - val_mean_absolute_error: 1.1750
Epoch 2/10


   0/1000 [..............................] - 0s 0s/step - loss: 2.7168 - mean_absolute_error: 1.2462 - val_loss: 2.2847 - val_mean_absolute_error: 1.1750


69.34279108047485

In [8]:
start = time()
metric = model.evaluate(test)
time() - start

1000/1000 [==============================] - 9s 9ms/step - loss: 2.2847 - mean_absolute_error: 1.1750


9.540741682052612

In [9]:
metric

[2.2847254276275635, 1.1749879121780396]